In [1]:
# default_exp gui.capture_command

# Capture Command

> Capture a user command with a tk GUI.

TODO: abstract out win32 bits

In [1]:
#export
from tkinter import *
from tkinter import ttk
from collections import namedtuple
from PIL import Image, ImageTk
import win32api, win32con, time

https://docs.microsoft.com/en-us/windows/win32/api/winuser/nf-winuser-mouse_event

In [2]:
#export
def pointer_left_click(x=10, y=10):
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, x, y)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, x, y)
    
def pointer_right_click(x=10, y=10):
    win32api.mouse_event(win32con.MOUSEEVENTF_RIGHTDOWN, x, y)
    win32api.mouse_event(win32con.MOUSEEVENTF_RIGHTUP, x, y)

In [3]:
#export
def get_pointer_position():
    return win32api.GetCursorPos()

In [4]:
#export 
def get_screen_size():
    return win32api.GetSystemMetrics(0), win32api.GetSystemMetrics(1)

In [5]:
#export
def pointer_in_left_half_of_screen():
    screen_width, _ = get_screen_size()
    cursor_x, _ = get_pointer_position()
    return cursor_x < (screen_width / 2)

In [6]:
#export
class TkKeyPressHandler:
    def __init__(self, root):
        self.root, self.keys_pressed, self.previous_after_id = root, [], None
    def key_pressed(self, event):
        self.keys_pressed.append(event.keysym)
        if self.previous_after_id is not None:
            self.root.after_cancel(self.previous_after_id)
        self.previous_after_id = self.root.after(200, self.root.destroy) # TODO: 200 config - how long do we wait for double press?

In [7]:
#export
def capture_key_press(side_bar_text=[], side_bar_left=True):
    screen_width, screen_height = get_screen_size()
    
    side_bar_column, img_column = (0, 1) if side_bar_left else (1, 0)
    
    root = Tk()
    mainframe = ttk.Frame(root, padding=5)
    mainframe.grid(column=0, row=0)

    # hack to force column widths
    ttk.Frame(mainframe, width=250-10).grid(column=side_bar_column, row=0)
    ttk.Frame(mainframe, width=screen_width-250).grid(column=img_column, row=0)
    
    side_bar_frame = ttk.Frame(mainframe, padding=5)
    side_bar_frame.grid(column=side_bar_column, row=0)
    for row, text in enumerate(side_bar_text):
        ttk.Label(side_bar_frame, text=text, font='TkFixedFont').grid(column=0, row=row, sticky=W)
    
    image = ImageTk.PhotoImage(Image.new('1', (screen_width-250,screen_height-10), color = 'white'))
    ttk.Label(mainframe, image=image).grid(column=img_column, row=0)
    
    root.attributes('-alpha', 0.9) # we can easily make the whole thing semi-transparent
    root.attributes('-transparentcolor', 'white') # turns image into a "window"
    root.attributes("-topmost", True)
    root.attributes('-fullscreen', True)
    
    def _destroy(event=None): root.destroy()
    root.bind("<Escape>", _destroy)
    root.bind("<FocusOut>", _destroy)
    
    key_press_handler = TkKeyPressHandler(root)
    root.bind('<Key>', key_press_handler.key_pressed)
    
    root.after(10, pointer_left_click) # make sure we have focus
    root.mainloop()
    return key_press_handler.keys_pressed

In [8]:
#export
Command = namedtuple('Command', ['key', 'label', 'fn'])
commands = [
    Command('l', 'Left click', lambda: pointer_left_click(*get_pointer_position())),
    Command('r', 'Right click', lambda: pointer_right_click(*get_pointer_position())),
    Command('semicolon', 'Enter / Return', lambda: win32api.keybd_event(win32con.VK_RETURN, 0)),
    Command('e', 'End', lambda: win32api.keybd_event(win32con.VK_END, 0)),
    Command('h', 'Home', lambda: win32api.keybd_event(win32con.VK_HOME, 0)),
    Command('d', 'Delete', lambda: win32api.keybd_event(win32con.VK_DELETE, 0)),
    Command('b', 'Backspace', lambda: win32api.keybd_event(win32con.VK_BACK, 0)),
    Command('as', 'Test seq', lambda: print('"as" sequence hit')),
]
command_map = {c.key: c for c in commands}

In [9]:
#do_not_test
keys_pressed = capture_key_press([f"{c.key}: {c.label}" for c in commands], not pointer_in_left_half_of_screen())
keys_pressed

['h']

In [10]:
#export
def run_command(command_map, keys_pressed):
    if len(keys_pressed) == 0:
        print('No keys pressed - nothing to do')
        return
    # TODO: do we want to limit the number of keys pressed?
#     if len(keys_pressed) > 2: 
#         print(f'Too many keys pressed. expected 1 or 2 but found {len(keys_pressed)}')
#         return
    k = ''.join(keys_pressed)
    if k in command_map:
        command_map[k].fn()
        return
    for k in keys_pressed: 
        if k in command_map:
            command_map[k].fn()
            time.sleep(0.1) # some key presses (backspace) might be ignored if they happen to fast
        else:
            print(f'{k} not mapped to a command')

In [11]:
run_command(command_map, ['a','s'])

"as" sequence hit


In [13]:
#do_not_test
time.sleep(2)
keys_pressed = capture_key_press([f"{c.key}: {c.label}" for c in commands], not pointer_in_left_half_of_screen())
print('keys_pressed', keys_pressed)
run_command(command_map, keys_pressed)

keys_pressed ['h']


In [14]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 05a_ml_data.ipynb.
Converted 05b_ml_model.ipynb.
Converted 10a_mediapipe_face_mesh_identify_landmarks.ipynb.
Converted 10b_mediapipe_face_mesh_capture_session.ipynb.
Converted 10c_mediapipe_face_mesh_train_model.ipynb.
Converted 10d_test_np_model.ipynb.
Converted 20a_gui_capture_command.ipynb.
Converted 70_cli.ipynb.
Converted index.ipynb.
Converted project_lifecycle.ipynb.
